In [2]:
import torch
# 导入 PyTorch 的负对数似然损失函数（Negative Log Likelihood Loss）。虽然在这段代码中没有使用，但它和信息量（self-information）以及对数概率的概念相关。
from torch.nn import NLLLoss

# 作用：对所有有效的数字进行求和
def nansum(x):
    # torch.isnan(x) 返回一个布尔张量，标记 x 中哪些位置是 NaN（Not a number非数字）。
    # ~torch.isnan(x) 对布尔张量取反，即只保留是数字的位置。
    # x[~torch.isnan(x)]：过滤掉 NaN 元素，只留下有效数值。
    return x[~torch.isnan(x)].sum()

# 自信息量等于 概率的负对数
def self_information(p):
    return -torch.log2(torch.tensor(p)).item()

print(self_information(1 / 64))


def entropy(p):
    entropy = - p * torch.log2(p)
    # Operator `nansum` will sum up the non-nan number
    out = nansum(entropy)
    return out

entropy(torch.tensor([0.1, 0.5, 0.1, 0.3]))

# 这里的代码其实和entropy一模一样，只是在数学上对两个随机变量的联合概率分布有新的解释。
def joint_entropy(p_xy):
    joint_ent = -p_xy * torch.log2(p_xy)
    # Operator `nansum` will sum up the non-nan number
    out = nansum(joint_ent)
    return out

# H(X,Y) = -(0.1 log2 0.1 + 0.5 log2 0.5 + 0.1 log2 0.1 + 0.3 log2 0.3)
joint_entropy(torch.tensor([[0.1, 0.5],
                            [0.1, 0.3]]))


def conditional_entropy(p_xy, p_x):
    p_y_given_x = p_xy/p_x
    print("p_y_given_x")
    print(p_y_given_x)
    cond_ent = -p_xy * torch.log2(p_y_given_x)
    # Operator `nansum` will sum up the non-nan number
    out = nansum(cond_ent)
    return out

# 这里的概率分布有点奇怪
conditional_entropy(torch.tensor([[0.1, 0.5], [0.2, 0.3]]),
                    torch.tensor([0.2, 0.8]))



def mutual_information(p_xy, p_x, p_y):
    p = p_xy / (p_x * p_y)
    mutual = p_xy * torch.log2(p)
    # Operator `nansum` will sum up the non-nan number
    out = nansum(mutual)
    return out

mutual_information(torch.tensor([[0.1, 0.5], [0.1, 0.3]]),
                   torch.tensor([0.2, 0.8]), torch.tensor([[0.75, 0.25]]))



# KL散度：
def kl_divergence(p, q):
    kl = p * torch.log2(p / q)
    out = nansum(kl)
    # 这里里有问题，因为如果p,q归一化了，它自然回事大于等于零的
    return out.abs().item()

6.0
p_y_given_x
tensor([[0.5000, 0.6250],
        [1.0000, 0.3750]])



下面关于对称性的实验种用到了：
$\text{Relative Difference} = \frac{|a - b|}{\tfrac{|a| + |b|}{2}}$

In [5]:
# KL散度例子：
# 生成并排序三个长度为 10,000 的张量：
#     p：服从标准正态分布 N(0,1)。
#     q1：服从 N(−1,1)。
#     q2：服从 N(1,1)。

# torch.manual_seed(seed) 会设置 当前 Python 进程内 的 CPU 和默认 GPU（如果有的话） 的随机数生成器（RNG）的种子。
torch.manual_seed(1)

tensor_len = 10000
p = torch.normal(0, 1, (tensor_len, ))
q1 = torch.normal(-1, 1, (tensor_len, ))
q2 = torch.normal(1, 1, (tensor_len, ))

p = torch.sort(p)[0]
q1 = torch.sort(q1)[0]
q2 = torch.sort(q2)[0]

# 对称性实验
# q1和q2关于x=0对称；预期p与q1的散度 与 p与q2的散度接近
kl_pq1 = kl_divergence(p, q1)
kl_pq2 = kl_divergence(p, q2)
similar_percentage = abs(kl_pq1 - kl_pq2) / ((kl_pq1 + kl_pq2) / 2) * 100

print(kl_pq1, kl_pq2, similar_percentage)
# 输出: (8582.034..., 8828.309..., 2.83...)

# 不对称性测试
# 对比q2与p的散度 与 p与q2的散度：
kl_q2p = kl_divergence(q2, p)
differ_percentage = abs(kl_q2p - kl_pq2) / ((kl_q2p + kl_pq2) / 2) * 100

print(kl_q2p, differ_percentage)
# 输出: (14130.125, 46.18...)


8582.0341796875 8828.3095703125 2.8290698237936858
